In [1]:
import pandas as pd

In [2]:
headers = pd.read_csv('../Data/handles-data.csv')
tweets = pd.read_csv('../Data/tweets-data.csv')
union = pd.read_csv('../Data/union.csv', sep=";")
pd.options.display.float_format = '{:.2f}'.format

/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Target

El target de nuestro proyecto va a ser el campo **nretweets** (número de Retweets).

He decidido usar este campo por su posible valor para conocer si el tuit va a tener la repercusión deseada o no.

He llegado a este conclusión después de ver cómo se comporta dependiendo de la hora y del día de la semana. Aunque no existen variaciones muy grandes, sí que he podido comprobar que ciertas horas y ciertos días hay más inteacción dentro de la red social.
Donde se puede ver que los usuarios dan más a me gusta y retuitean. Por lo que seguramente exista un patrón en los datos, dado un patrón de conducta humana. Por ejemplo, en las horas de "sueño" existe menos interacción.



# Limpieza de los Datos

## Eliminar No interesantes

In [3]:
print(tweets.columns.tolist())

['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place', 'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url', 'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest']


In [24]:
datosInteresantes = union[['day','hour','retweet','nlikes']]
target = union[['nretweets']]
print(datosInteresantes.count())
print(target.count())

day        40867
hour       40867
retweet    40867
nlikes     40867
dtype: int64
nretweets    40867
dtype: int64


### Nulos

In [5]:
datosInteresantes.isnull().sum()

day         0
hour        0
retweet     0
nreplies    0
nlikes      0
dtype: int64

### Separamos datos categóricos

In [6]:
datosInteresantes.dtypes

day         int64
hour        int64
retweet      bool
nreplies    int64
nlikes      int64
dtype: object

El campo retweet es categórico (Verdadero o Falso)

In [7]:
datosInteresantes['retweet']

0         True
1        False
2         True
3        False
4        False
         ...  
40862    False
40863    False
40864    False
40865    False
40866    False
Name: retweet, Length: 40867, dtype: bool

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [25]:
df_c = datosInteresantes.copy(deep = True)
enc_ohe = OneHotEncoder()
enc_ohe.fit(df_c[["retweet"]])

OneHotEncoder()

In [26]:
data_encoder = enc_ohe.transform(df_c[['retweet']]).toarray()
print(data_encoder)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


Lo unimos al dataset

In [27]:
df_encoder = pd.DataFrame(data_encoder)
df_encoder.head()
df_c = df_c.join(df_encoder,lsuffix='_left', rsuffix='_right')
df_c.drop(['retweet'], axis=1, inplace=True)


In [28]:
df_c

,day,hour,nlikes,0,1
0,4,0,0,0.00,1.00
1,2,22,5266,1.00,0.00
2,2,18,0,0.00,1.00
3,2,18,5638,1.00,0.00
4,2,7,8810,1.00,0.00
...,...,...,...,...,...
40862,2,13,961,1.00,0.00
40863,2,13,224,1.00,0.00
40864,2,13,240,1.00,0.00
40865,2,13,1357,1.00,0.00


## Separar datos Entrenamiento

In [13]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(df_c, target, test_size=0.3)

## Nulos

In [15]:
X_train.isnull().sum()

day         0
hour        0
nreplies    0
nlikes      0
0           0
1           0
dtype: int64

# Entrenar Modelo

In [16]:

import numpy as np
from sklearn.linear_model import LinearRegression

In [30]:
np.random.seed(5)


In [31]:
model=LinearRegression()


In [19]:
# X_train = np.asarray(X_train).astype(np.int64)

In [32]:
model.fit(X_train, y_train) 

/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


LinearRegression()

In [34]:

predictions = model.predict(X_test)
X_tested = X_test.copy()
print(X_tested.shape)
print(predictions.shape)
X_tested['predictions'] = predictions
X_tested['y'] = y_test

# X_tested['difference'] = (y_test - X_tested['predictions'])
X_tested
# X_test['predictions'] = predictions
# X_test['defference'] = (y_test) - (X_test.predictions)

# día,hora,nreplies,nlikes,0_retweet,1_retweet
# print(model.predict([[6,3,0,0,1,0]]))

(12261, 5)
(12261, 1)


/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,day,hour,nlikes,0,1,predictions,y
40592,6,9,2676,1.00,0.00,476.96,438
4773,3,17,0,0.00,1.00,586.20,30
10397,4,14,0,0.00,1.00,577.38,4
29106,6,23,50,1.00,0.00,-177.60,35
1801,2,23,3034,1.00,0.00,598.46,543
...,...,...,...,...,...,...,...
1409,6,18,18,1.00,0.00,-181.58,8
19027,5,3,0,0.00,1.00,574.74,20
13334,1,21,56,1.00,0.00,-118.90,3
22346,1,19,36,1.00,0.00,-122.26,5


In [22]:
X_tested.describe()

,day,hour,nreplies,nlikes,0,1,predictions
count,12261.00,12261.00,12261.00,12261.00,12261.00,12261.00,12261.00
mean,3.83,13.04,79.43,1395.66,0.85,0.15,293.72
std,1.89,7.30,924.95,9519.79,0.36,0.36,2340.23
min,1.00,0.00,0.00,0.00,0.00,0.00,-190.21
25%,2.00,6.00,0.00,5.00,1.00,0.00,-151.11
50%,4.00,15.00,3.00,38.00,1.00,0.00,-127.26
75%,5.00,19.00,24.00,205.00,1.00,0.00,217.37
max,7.00,23.00,88035.00,452062.00,1.00,1.00,109726.51
